In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymupdf4llm
from pymupdf4llm import LlamaMarkdownReader

In [22]:
md_read = LlamaMarkdownReader()
data = md_read.load_data(r"C:\Users\akers\OneDrive\Documenten\GitHub\Datalab-V\Training_set_text\European_policies\CELEX_32018L2001_EN_TXT.pdf")

Successfully imported LlamaIndex
Processing C:\Users\akers\OneDrive\Documenten\GitHub\Datalab-V\Training_set_text\European_policies\CELEX_32018L2001_EN_TXT.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing C:\Users\akers\OneDrive\Documenten\GitHub\Datalab-V\Training_set_text\European_policies\CELEX_32018L2001_EN_TXT.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing C:\Users\akers\OneDrive\Documenten\GitHub\Datalab-V\Training_set_text\European_policies\CELEX_32018L2001_EN_TXT.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing C:\Users\akers\OneDrive\Documenten\GitHub\Datalab-V\Training_set_text\European_policies\CELEX_32018L2001_EN_TXT.pdf...
[                                        ]

In [35]:
print(data[4])

Doc ID: 7ec87f1e-7ec0-4f33-a4f6-8bd1e80b0c40
Text: L 328/86 EN Official Journal of the European Union 21.12.2018
(24) In order to ensure that the opening of support schemes is
reciprocal and brings mutual benefits, cooperation agreements should
be signed between participating Member States. Member States should
retain control over the pace of deployment of renewable electricity
capacity on thei...
